# NLP Project Part 1 


# data préparation 

In [ ]:
import pandas as pd
import numpy as np
import spacy
import tqdm 
#instantiating English module
nlp = spacy.load('en')

In [ ]:
setement=pd.read_csv('data.csv') 
setement.head()  

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral


We transform Sentiment into "int"  Data 🇰 

In [ ]:
for index,raw in setement.iterrows():  
    if raw['Sentiment'] == 'positive' : 
      setement['Sentiment'][index]=1 
    if  raw['Sentiment'] == 'negative' : 
      setement['Sentiment'][index]=-1   
    if  raw['Sentiment'] == 'neutral' :  
      setement['Sentiment'][index]=0  


In [ ]:
setement.head()

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,1
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",-1
2,"For the last quarter of 2010 , Componenta 's n...",1
3,According to the Finnish-Russian Chamber of Co...,0
4,The Swedish buyout firm has sold its remaining...,0


In [ ]:
setement['Sentence'][0] 

"The GeoSolutions technology will leverage Benefon 's GPS solutions by providing Location Based Search Technology , a Communities Platform , location relevant multimedia content and a new and powerful commercial model ."

**We import tools to clean text **

In [ ]:
%%time
nlp(setement['Sentence'][1])  

CPU times: user 19.8 ms, sys: 931 µs, total: 20.7 ms
Wall time: 29.6 ms


$ESI on lows, down $1.50 to $2.50 BK a real possibility

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords") 
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import re  
import string

We clean all sentence in our data frame :

In [ ]:
%%time
clean_sen = []


pbar = tqdm.tqdm(total=len(setement['Sentence']),position=0, leave=True)

for text in nlp.pipe(setement['Sentence'], disable=["tagger", "parser", "ner"]):

  txt = [token.lemma_.lower() for token in text 
         if token.is_alpha 
         and not token.is_stop 
         and not token.is_punct]

  clean_sen.append(txt)

  pbar.update(1)



 99%|█████████▉| 5771/5842 [00:01<00:00, 5114.68it/s]

CPU times: user 1.28 s, sys: 26.3 ms, total: 1.31 s
Wall time: 1.33 s


WE replace sentecnce with a list of words ✅ 

In [ ]:
setement['Sentence']=clean_sen 

In [ ]:
setement.head()    

,Sentence,Sentiment
0,"[geosolutions, technology, leverage, benefon, ...",1
1,"[esi, low, bk, real, possibility]",-1
2,"[quarter, componenta, net, sale, double, be, b...",1
3,"[according, finnish, russian, chamber, commerc...",0
4,"[swedish, buyout, firm, sell, remain, percent,...",0


In [ ]:
print(setement['Sentence'].isnull().sum() , 'NO NULL great day' ) 

0 NO NULL great day


We install the packages which we need to transform the words to vectors as Word2vec: 

In [ ]:
!pip install --upgrade gensim -q

     |████████████████████████████████| 24.1 MB 54.3 MB/s 


In [ ]:
import logging 
from gensim.models import Word2Vec 
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO) 

WE transform each word to vector of 200 element and we count word which are more then 10  ⚔

In [ ]:
w2v_model = Word2Vec(sentences=setement['Sentence'], vector_size=200, window=5, min_count=10, workers=2, epochs=10) 

2022-03-13 20:30:14,056 : INFO : collecting all words and their counts
2022-03-13 20:30:14,060 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-03-13 20:30:14,085 : INFO : collected 8176 word types from a corpus of 62135 raw words and 5842 sentences
2022-03-13 20:30:14,087 : INFO : Creating a fresh vocabulary
2022-03-13 20:30:14,101 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=10 retains 1108 unique words (13.551859099804306%% of original 8176, drops 7068)', 'datetime': '2022-03-13T20:30:14.101571', 'gensim': '4.1.2', 'python': '3.7.12 (default, Jan 15 2022, 18:48:18) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.144+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'prepare_vocab'}
2022-03-13 20:30:14,104 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=10 leaves 46932 word corpus (75.53230868270701%% of original 62135, drops 15203)', 'datetime': '2022-03-13T20:30:14.104061', 'gensim': '4.1.2', 'python': '3.7.12 (default, Jan 15 2022, 

In [ ]:
w2v_model.wv.similar_by_word('finance') #similarty to finance 

[('partnership', 0.9983425140380859),
 ('way', 0.9983341693878174),
 ('know', 0.9982154369354248),
 ('recycle', 0.9979555010795593),
 ('germany', 0.9977225065231323),
 ('responsible', 0.997700572013855),
 ('buyout', 0.99757319688797),
 ('advance', 0.9974996447563171),
 ('upgrade', 0.9974631667137146),
 ('competition', 0.997349739074707)]

In [ ]:
w2v_model.wv.similar_by_word('good') #similartiy to good  

[('near', 0.9981837272644043),
 ('like', 0.9975825548171997),
 ('look', 0.9969451427459717),
 ('signal', 0.9968292117118835),
 ('actual', 0.996150016784668),
 ('clear', 0.9958698749542236),
 ('think', 0.9958350658416748),
 ('get', 0.995799720287323),
 ('setup', 0.995729923248291),
 ('car', 0.9954541921615601)]

In [ ]:
w2v_model.wv.similar_by_word('low') #similartiy to good  

[('pct', 0.9471407532691956),
 ('turnover', 0.9287186861038208),
 ('bln', 0.9285021424293518),
 ('october', 0.9227984547615051),
 ('end', 0.9222664833068848),
 ('percent', 0.9185442924499512),
 ('see', 0.9164886474609375),
 ('warn', 0.9158691763877869),
 ('fiscal', 0.9089294075965881),
 ('trade', 0.9077963829040527)]

 WE take avrege of each vector of word to make one vectors of each sentence in our Data frame : with this function that i found it in teacher code  ☝

In [ ]:
def get_mean_vector(w2v_model, words):
    # remove out-of-vocabulary words
    words = [word for word in words if word in w2v_model.wv.key_to_index]
    if len(words) >= 1:
        return np.mean(w2v_model.wv[words], axis=0) 
    else:
        return [] 

In [ ]:
w2v_model.save('w2v_model') 

2022-03-13 20:30:15,104 : INFO : Word2Vec lifecycle event {'fname_or_handle': 'w2v_model', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2022-03-13T20:30:15.098881', 'gensim': '4.1.2', 'python': '3.7.12 (default, Jan 15 2022, 18:48:18) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.144+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'saving'}
2022-03-13 20:30:15,110 : INFO : not storing attribute cum_table
2022-03-13 20:30:15,147 : INFO : saved w2v_model


In [ ]:
avg_vecs =setement['Sentence'].map(lambda t: get_mean_vector(w2v_model, t))   

In [ ]:
avg_vecs

0       [-0.023638105, -0.2225693, 0.05494887, -0.0885...
1       [-0.049470887, -0.18776272, -0.13305257, 0.161...
2       [0.11238689, 0.04761131, -0.22016947, 0.495715...
3       [-0.010964988, -0.17172924, 0.012902707, 0.039...
4       [0.012547172, -0.10590384, -0.07147517, 0.1087...
                              ...                        
5837    [0.0020562652, -0.12607355, -0.009878085, -0.0...
5838    [-0.0032976754, -0.12049678, -0.025588552, 0.0...
5839    [0.0036183472, -0.10471592, -0.022085711, 0.03...
5840    [0.07394402, 0.033335775, -0.16605195, 0.35318...
5841    [-0.036986586, -0.21293043, -0.016931655, 0.09...
Name: Sentence, Length: 5842, dtype: object

In [ ]:
len(avg_vecs)

5842

Aaaaaah there is problem some vectors are null  
so we delete them esay ❎ 

In [ ]:
len(avg_vecs[29])==0  

True

In [ ]:
to_del=[]
for i in range(0,5841) : 
   if  len(avg_vecs[i])==0 :  
       to_del.append(i) 

      

In [ ]:
to_del

[29,
 175,
 292,
 318,
 497,
 921,
 1081,
 1190,
 1191,
 1487,
 1637,
 1794,
 1991,
 2102,
 2115,
 2145,
 2474,
 2912,
 2987,
 3373,
 3630,
 3954,
 4005,
 4374,
 4720,
 4875,
 4955,
 5123,
 5345,
 5521,
 5545,
 5693,
 5746,
 5811]

In [ ]:
for i in to_del : 
  del avg_vecs[i]  

We trosform our vector to a matrix : 

In [ ]:
avg_vecs = np.vstack(avg_vecs)  

In [ ]:
avg_vecs.shape  

(5808, 200)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

We virefey similarty with Row FIVE 🐾 

In [ ]:
sims = cosine_similarity(avg_vecs[4].reshape(1,200), avg_vecs)   

In [ ]:
# extract indices
ix = np.flip(np.argsort(sims)).tolist()[0][:10]  

In [ ]:
setement['Sentence'][ix]  

4       [swedish, buyout, firm, sell, remain, percent,...
126     [tyc, break, new, high, lot, technicals, look,...
2509         [crh, concrete, bid, holcim, lafarge, asset]
5505    [incap, contract, manufacturing, carry, manufa...
4237    [financial, report, publish, friday, seb, say,...
5685    [time, finnair, award, agreement, uk, government]
3011               [upset, machinery, work, take, poland]
3660    [renesas, mobile, europe, ltd, approximately, ...
1427                 [result, taxis, loss, million, euro]
570       [operating, profit, decrease, eur, mn, eur, mn]
Name: Sentence, dtype: object

## Some query :

In [ ]:
query = 'good year'

query = nlp(query, disable=["tagger", "parser", "ner"])
query = [token.lemma_.lower() for token in query 
         if token.is_alpha 
         and not token.is_stop 
         and not token.is_punct]

query = get_mean_vector(w2v_model,query)

In [ ]:
sims = cosine_similarity(query.reshape(1,200), avg_vecs) 
ix = np.flip(np.argsort(sims)).tolist()[0][:10]
setement['Sentence'][ix]  

2410    [competition, authority, approve, deal, finalize]
1469    [skf, april, alandsbanken, give, buy, recommen...
2995    [delhaize, le, lion, select, aldata, vocal, pd...
4024    [company, see, net, profit, quarter, be, be, c...
1132                         [public, service, available]
4060    [previously, order, set, start, produce, elect...
1342                  [company, say, measure, long, need]
4884    [nokia, music, store, begin, trade, tuesday, s...
1648    [aldata, say, numb, operational, aspect, defin...
16      [msft, sql, server, revenue, grow, double, dig...
Name: Sentence, dtype: object